In [66]:
import xpress as xp
import pandas as pd
import math
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np

In [72]:
# Read the Excel file, turn into dataframe
pool = pd.read_excel('SampleData.xlsx', engine='openpyxl')

# Remove unnecessary coloumns
df = pool.drop('dage', axis=1)

In [73]:
edges = df.dropna(subset=['score']).set_index(['donor_id', 'recipient'])['score'].to_dict()

In [74]:
# Parameters for the optimization:
k = 4 # max cycle length

In [75]:
# Input data from the dataset
# Ensure compatibility with Example input data below

# This lists all donor id's that are not altruistic donors
pairs = list(set(df[df['altruistic'] != True]['donor_id']))
pairs = [float(i) for i in pairs]

 
altruistic_donors = list(set(df[df['altruistic'] == True]['donor_id']))
altruistic_donors = [float(i) for i in altruistic_donors]

nodes = pairs + altruistic_donors

edges = edges

In [76]:
# # Example input data 1

# pairs = ["P1", "P2", "P3", "P4"] 
# altruistic_donors = ["NDD1"]
# nodes = pairs + altruistic_donors
# edges = {("NDD1", "P1"): 2,
#          ("P1", "P2"): 10, 
#          ("P2", "P3"): 10,
#          ("P3", "P4"): 10,
#          ("P4", "P1"): 10
# }

# # Example input data 2

# pairs = ["P1", "P2", "P3", "P4", "P5"] 
# altruistic_donors = ["NDD1"]
# nodes = pairs + altruistic_donors
# edges = {("NDD1", "P1"): 0.1,
#          ("P1", "P2"): 10, 
#          ("P2", "P3"): 9,
#          ("P3", "P4"): 8,
#          ("P4", "P5"): 7,
#          ("P5", "P1"): 6
# }

# # Example input data 3

# pairs = ["P1", "P2", "P3", "P4", "P5"] 
# altruistic_donors = ["NDD1"]
# nodes = pairs + altruistic_donors
# edges = {("NDD1", "P1"): 0.1,
#          ("P1", "P2"): 10, 
#          ("P2", "P3"): 9,
#          ("P3", "P4"): 8,
#          ("P4", "P5"): 7
# }

In [77]:
# Create the loop!

# Create Xpress Model
# Initialize the model
prob = xp.problem()

# Define decision variables for each edge
y = {e: xp.var(vartype=xp.binary, name=f"y_{e[0]}_{e[1]}") for e in edges}
prob.addVariable(list(y.values()))

# Objective: Maximize total benefit
prob.setObjective(xp.Sum(y[e] * w for e, w in edges.items()), sense=xp.maximize)

# Constraints
for v in pairs:
    prob.addConstraint(xp.Sum(y[e] for e in edges if e[0] == v) <= xp.Sum(y[e] for e in edges if e[1] == v))
    prob.addConstraint(xp.Sum(y[e] for e in edges if e[1] == v) <= 1)

for a in altruistic_donors:
    prob.addConstraint(xp.Sum(y[e] for e in edges if e[0] == a) <= 1)

finished = False # A flag to mark the end of the optimization.
infeasible = False # A (currently unused) flag to mark no feasible solution.
# TODO: Add a catch for no feasible solution. This shouldn't happen but it is
# probably better to be robust about this.

while finished == False and infeasible == False:
  
    # Solve the model
    prob.solve()
    opt_sol = prob.getSolution()

    # Construct the graph from the optimal solution:
    DG = nx.DiGraph()
    selected_edges = [list(edges.keys())[i] for i, e in enumerate(list(edges.keys())) if opt_sol[i]==1]
    DG.add_edges_from(selected_edges)

    # Check if there is a cycle length that is too long:
    cycles = list(nx.simple_cycles(DG))

    # If ok, report done.
    # TODO: Rewrite this so that it is with the max_cycle bit...
    if cycles==[] or max(map(len,cycles))<=k:
        print("")
        print("##########################################################")
        print(f"OPTIMIZATION COMPLETE: no cycles of length more than {k}.")
        print("##########################################################")
        print("")
        finished = True
        break
    
    # If not done, report that reoptimization is required:
    else:
        print("")
        print("#################################################################")
        print(f"REOPTIMIZATION REQUIRED: proposed solution contains long cycles.")
        print("#################################################################")
        print("")

    # Take the long cycle we found and make a note of its edges:
    max_cycle = max(cycles,key=len)
    cycle_edges = [(max_cycle[i],max_cycle[i+1]) for i in range(len(max_cycle)-1)]
    cycle_edges += [(max_cycle[-1],max_cycle[0])]

    # Add the constraint to remove this as an option: 
    prob.addConstraint(xp.Sum(y[e] for e in cycle_edges) <= len(max_cycle)-1)


# Print the output
print("")
print("")
print("")

print("Optimal Matches:")
for (u, v), var in y.items():
    if prob.getSolution(var) > 0.5:
        print(f"{u} donates to {v} with benefit {edges[(u,v)]}")


print(f"Total Benefit: {prob.getObjVal()}")
print("")
print("")
print("")

FICO Xpress v9.4.2, Community, solve started 22:58:04, Feb 27, 2025
Heap usage: 472KB (peak 472KB, 217KB system)
Maximizing MILP noname using up to 16 threads and up to 15GB memory, with these control settings:
OUTPUTLOG = 1
NLPPOSTSOLVE = 1
XSLP_DELETIONCONTROL = 0
XSLP_OBJSENSE = -1
Original problem has:
       103 rows          261 cols          783 elements       261 entities
Presolved problem has:
        88 rows          235 cols          690 elements       235 entities
Presolve finished in 0 seconds
Heap usage: 556KB (peak 652KB, 217KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.00e+00,  1.00e+00] / [ 1.00e+00,  1.00e+00]
  RHS and bounds [min,max] : [ 1.00e+00,  1.00e+00] / [ 1.00e+00,  1.00e+00]
  Objective      [min,max] : [ 1.00e+00,  8.90e+01] / [ 1.00e+00,  1.21e+02]
Autoscaling applied standard scaling

Will try to keep branch and bound tree memory usage below 7.4GB
 *** Solution found:   194.00000


Performing root presolve...

Reduced problem has:      85 rows     162 columns       483 elements
Presolve dropped   :       5 rows      73 columns       218 elements
Will try to keep branch and bound tree memory usage below 433MB
 
   Its         Obj Value      S   Ninf  Nneg   Sum Dual Inf  Time
   110       1955.000000      D      0     0        .000000     0
Optimal solution found
Dual solved problem
  110 simplex iterations in 0.00 seconds at time 0

Final objective                       : 1.955000000000000e+03
  Max primal violation      (abs/rel) :       0.0 /       0.0
  Max dual violation        (abs/rel) :       0.0 /       0.0
  Max complementarity viol. (abs/rel) :       0.0 /       0.0

Starting root cutting & heuristics
Deterministic mode with up to 1 additional thread
 
 Its Type    BestSoln    BestBound   Sols    Add    Del     Gap     GInf   Time
b         1947.000000  1947.000000      2                 -0.00%       0      0
 *** Search completed ***
Uncrunching matri